This is an exploration of names with different dynamics: those that stay consistently popular through whole centuries and those that have sudden popularity spikes and then fade away quickly.

This is still under construction. Please drop a comment if this notebook sparks any interesting thoughts or questions.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Load data and compute some statistics on names.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
df = pd.read_csv('../input/NationalNames.csv', index_col='Id')

names = pd.DataFrame()
#print(names.head()) prints emty data frame as names is 
gr = df.groupby(['Name','Gender'])
names['Total'] = gr['Count'].sum()
names['Max'] = gr['Count'].max()
def bestyear(group):
    years, counts = group['Year'].values, group['Count'].values
    ind = np.argmax(counts)
    return years[ind]
names['BestYear'] = gr.apply(bestyear)
names['FirstYear'] = gr['Year'].first()
names['LastYear'] = gr['Year'].last()

## Contrasting dynamics

Two contrasting girl names: "Delia" stays consistenly popular through the years while "Deneen" has a sudden popularity spike and then fades quickly.

In [ ]:
fig, ax = plt.subplots()
data = df.loc[(df.Name=='Delia')&(df.Gender=='F')]
ax.bar(data.Year, data.Count, width = 1, alpha=0.6, label='Delia', color='orange')
data = df.loc[(df.Name=='Deneen')&(df.Gender=='F')]
ax.bar(data.Year, data.Count, width = 1, alpha=0.6, label='Deneen', color='r')
ax.legend();

## A measure of "suddenness"

In order to find other names with spike-fade dynamics I'll look at a measure calculated by dividing a name's maximum count by its total count. This will be a number ranging from 0 to 1 that may be thought of as a measure of "suddenness".

In [ ]:
names['Sudden'] = names.Max.values/names.Total.values
print('"Suddenness" score')
print('{}: {:.3f}'.format('Delia',names.loc[('Delia','F'),'Sudden']))
print('{}: {:.3f}'.format('Deneen',names.loc[('Deneen','F'),'Sudden']))

## Most consistent names

Now let's filter our names list down to names that have a count of at least 1000 and look at names that stayed consistently popular through the years.

In [ ]:
popular = names.loc[names.Total>1000]\
               .sort_values(by='Sudden')

In [ ]:
def plot(ax, name, gender):
    data = df.loc[(df.Name==name)&(df.Gender==gender)]
    ax.bar(data.Year, data.Count, width = 1, alpha=0.6,
           color = 'r' if gender=='F' else 'b')
    xlow, xhigh = ax.get_xlim()
    ylow, yhigh = ax.get_ylim()
    ax.set_title(name)
    return ax

fig, ax = plt.subplots(4,4, figsize=(12,12), sharex=True)
ax = ax.flatten()
for a, (name, gender) in zip(ax, popular.head(16).index):
    plot(a, name, gender)
fig.suptitle('Names with persistent popularity',fontsize='x-large');

- More male names than female ones here
- French, spanish or italian names like Domingo, Teodoro, Maurice, Alonzo.

## Most sudden names

Now let's look at the flash-like names.

In [ ]:
popular = popular.sort_values(by='Sudden',ascending=False)

In [ ]:
fig, ax = plt.subplots(4,4, figsize=(12,12), sharex=True)
ax = ax.flatten()
for a, (name, gender) in zip(ax, popular.head(16).index):
    plot(a, name, gender)
fig.suptitle('Names with sudden popularity flashes',fontsize='x-large');

- A lot of female names here. Are boy names more conservative than girl names? Compute a weighted average of suddenness score for M and F names.
- For some chosen names we don't see the fade part because their peak year is the last one in the dataset.

## A look at spike-fade names

We select names with high Sudden scores and having best year before 2010 so that we know there is a fade part.

In [ ]:
flashy = popular.loc[(popular.Sudden > 0.2)&(popular.BestYear < 2010)]
gender = flashy.index.get_level_values(1)
c=np.array(['b']*flashy.shape[0])
c[np.where(gender=='F')]='r'
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(flashy.BestYear, np.arange(flashy.shape[0]),c=c,s=0.1*flashy.Total, alpha=0.5)
ax.scatter(flashy.BestYear, np.arange(flashy.shape[0]),c=c,s=0.1*flashy.Max, alpha=0.5)
np.random.seed(0)
for (n,i) in enumerate(flashy.index):
    #ax.plot([flashy.loc[i,'FirstYear'],flashy.loc[i,'LastYear']],
    #[n,n],'b-' if i[1]=='M' else 'r-')
    right = np.random.choice([-1,1])
    ax.text(flashy.loc[i,'BestYear']+right, n, i[0],
            verticalalignment='center',
            horizontalalignment = 'left' if right==1 else 'right')
ax.set_ylim(-2, ax.get_ylim()[1])
ax.set_yticklabels([]);
ax.set_title('Flashy names at a glance');
print('inner circle: number of babies this year, \nouter circle: total count')

- More names with popularity flashes in recent years
- Alternate spellings of the same name flash together (see Katina and Catina; Miley, Mylie and Mylee; Jaslene and Jazlene)

## Flash causes

I googled around for likely reasons of popularity spikes of the names in the chart above. The searches mostly reveal TV characters, popular musicians, movie actors, people from reality shows and sport stars. See below for who's who.

### Christop

Name 'Christop' only appeared for one year and only in one state (NY 1989). I think that it's likely a typo in the data and the name should be 'Cristopher'. There are 28 more names from NY 1989 that are missing their last letters so this probably indicates some data collection problems. See details [here](https://www.kaggle.com/dvasyukova/d/kaggle/us-baby-names/is-it-a-typo).

![Boys named Christopher in NY][1]


  [1]: https://www.kaggle.io/svf/336729/c78323d455d23a9756f05d3abad660e2/__results___files/__results___11_1.png

### Kizzy

It looks like this name was popularized by a character of 1977 miniseries '[Roots](https://en.wikipedia.org/wiki/Roots_(1977_miniseries))'. First release dates are 23 Jan 1977 - 30 Jan 1977 (shown by a black line on the plot). It looks like there is a 2016 remake of this series so the name might make a comeback.

Alternatively there was a 1976 BBC series [Kizzy](https://en.wikipedia.org/wiki/Kizzy_(TV_series)) but that one was released in UK and is a less likely explanation.

In [ ]:
ax = plot(plt.axes(), 'Kizzy','F')
ax.vlines(1977+23/365, 0,1200)
ax.set_xlim(1976,1990);

### Deneen

musical group from the 60's - August & Deneen (?)

[We go together](https://www.discogs.com/August-Deneen-We-Go-Together/release/5838179) release on 11 May 1968 - doesn't really fit the spike...

In [ ]:
ax = plot(plt.axes(), 'Deneen','F')

### Tennille

[Captain & Tennille](https://en.wikipedia.org/wiki/Captain_%26_Tennille) - a music husband-and-wife duo popular in the 1970s.

From the wiki:

> Over the next few years, Captain and Tennille released a string of hit
> singles mostly from their first two albums titled Love Will Keep Us
> Together (US #2, 1975) and Song Of Joy (US #9, 1976)...
>
> ...
>
> Such was the level of their popularity that they were given their own
> television variety show. The Captain and Tennille TV show aired from
> September 1976 to March 1977 on ABC.

In [ ]:
ax = plot(plt.axes(), 'Tennille','F')

### Katina, Catina

According to [this site](http://www.behindthename.com/name/katina) this name had a spike in popularity in America in 1972 when it was used for a newborn baby on the soap opera '[Where the Heart Is](https://en.wikipedia.org/wiki/Where_the_Heart_Is_(US_TV_series))'.

'Katina' is the spelling used in the wiki page and it was about twice as popular as the alternative starting with a C.

In [ ]:
ax = plot(plt.axes(), 'Catina','F')
plot(ax, 'Katina','F')
ax.set_xlim(1965,1985);
ax.set_title('Katina, Catina')

### Nakia

[Nakia](https://en.wikipedia.org/wiki/Nakia_(TV_series)) - a TV series about a Native American police officer in New Mexico. It aired from September 21 to December 28, 1974. Why there is a comeback after 1990 I don't know.

In [ ]:
ax = plot(plt.axes(), 'Nakia','M')

### Levar

[LeVar Burton](https://en.wikipedia.org/wiki/LeVar_Burton) is an american actor who played the young Kunta Kinte in the 1977 television miniseries Roots that already gave us the name Kizzy.

In [ ]:
fig,ax = plt.subplots(3,1,sharex=True,sharey=True)
plot(ax[0], 'Levar','M')
plot(ax[1], 'Kunta','M')
plot(ax[2], 'Kinte','M')
fig.suptitle('Levar - actor, Kunta Kinte - character');

Kizzy was played by [Leslie Uggams](https://en.wikipedia.org/wiki/Leslie_Uggams). Was there an effect on the name Leslie?

In [ ]:
ax = plot(plt.axes(), 'Leslie','F')
ax = plot(ax, 'Kizzy','F')
ax.set_title('Leslie - actress, Kizzy - character')
ax.set_xlim(1975,2015);

### Marely

Can't find any references for Marely. Is it a variant of Marley?

Marley also spiked in popularity in 2008.

[Marley & Me](https://en.wikipedia.org/wiki/Marley_%26_Me_%28film%29) is a 2008 American coming-of-age comedy-drama film about the titular dog, Marley. Why name kids after a dog I honestly don't know =).

It looks like Marely starts getting popular in 2007 - before the release of the film. Need to find some other reason. Or was a trailer enough?

In [ ]:
fig,ax = plt.subplots(3,1,sharex=True,sharey=False)
plot(ax[0], 'Marely','F')
plot(ax[1], 'Marley','F')
plot(ax[2], 'Marley','M')
ax[0].set_xlim(1980,2015);

### Kanye

[Kanye West](https://en.wikipedia.org/wiki/Kanye_West)  is an American recording artist, songwriter, record producer... Intent on pursuing a solo career as a rapper, West released his debut album The College Dropout in 2004 to widespread critical and commercial success (Wikipedia)

2004 fits with popularity spike.

In [ ]:
ax = plot(plt.axes(), 'Kanye','M')
ax.set_xlim(2000,2015)

### Shaquille

Probably after [Shaquille O'Neal](https://en.wikipedia.org/wiki/Shaquille_O'Neal), a star basketball player. According to wikipedia he won the NBA Rookie of the Year award in 92-93.

In [ ]:
ax = plot(plt.axes(), 'Shaquille','M')

### Moesha

[Moesha](https://en.wikipedia.org/wiki/Moesha) was an American sitcom series that aired on the UPN network from January 23, 1996, to May 14, 2001

In [ ]:
ax = plot(plt.axes(), 'Moesha','F')

### Allisson

[Allisson Lozz](https://en.wikipedia.org/wiki/Allisson_Lozz) - a mexican actress, singer and model. She is best known for her roles in the Mexican telenovelas ... En Nombre del Amor (2008-2009) as Paloma.

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,sharey=False)
plot(ax[0], 'Allisson','F')
plot(ax[1], 'Paloma','F')
ax[0].set_xlim(1975,2015);

If this person was indeed the inspiration for this name then we should see most Allissons appear in states where people are more likely to watch mexican TV series (southern states?)

In [ ]:
ds = pd.read_csv('../input/StateNames.csv')
d = ds.loc[ds.Name=='Allisson']\
      .groupby('State')['Count'].sum().sort_values(ascending=False)
d.head(10)

### Khadijah, Kadijah + Latifah

[Khadijah James](https://en.wikipedia.org/wiki/Khadijah_James), character in the television series [Living Single](https://en.wikipedia.org/wiki/Living_Single) that aired for five seasons on the FOX network from August 29, 1993, to January 1, 1998. She is portrayed by [Queen Latifa](https://en.wikipedia.org/wiki/Queen_Latifah). Queen Latifah released her debut album in 1989.

*check other spelling variants*

In [ ]:
fig,ax = plt.subplots(4,1,sharex=True,sharey=False)
plot(ax[0], 'Khadijah','F')
plot(ax[1], 'Kadijah','F')
plot(ax[2], 'Latifah','F')
plot(ax[3], 'Queen','F')
ax[0].set_xlim(1980,2015);

### Akeelah

[Akeelah and the Bee](https://en.wikipedia.org/wiki/Akeelah_and_the_Bee) - a 2006 movie that tells the story of Akeelah Anderson (Keke Palmer), an 11-year-old girl who participates in the Scripps National Spelling Bee.

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,sharey=False)
plot(ax[0], 'Akeelah','F')
plot(ax[1], 'Keke','F')
ax[0].set_xlim(1980,2015);

### Jaslene, Jazlene

[Jaslene González](https://en.wikipedia.org/wiki/Jaslene_Gonzalez) is a Puerto Rican model, television host, and winner of [Cycle 8](https://en.wikipedia.org/wiki/America's_Next_Top_Model_%28cycle_8%29) of America's Next Top Model that aired from February 28, 2007 to May 16, 2007.

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,sharey=False)
plot(ax[0], 'Jaslene','F')
plot(ax[1], 'Jazlene','F')

### Aaden

Aaden Gosselin, "Jon and Kate plus 8" 3rd born of the Gosselin sextuplets

In [ ]:
plot(plt.gca(),'Aaden','M')

### Talan

[Laguna Beach: The Real Orange County](https://en.wikipedia.org/wiki/Laguna_Beach:_The_Real_Orange_County) is an American reality television series that originally aired on MTV from September 28, 2004 until November 15, 2006. Talan Torriero is one of the show's cast.

In [ ]:
plot(plt.gca(),'Talan','M')

### Analia

[El Rostro de Analía](https://en.wikipedia.org/wiki/El_Rostro_de_Anal%C3%ADa) (English: The Face of Analía) is a Spanish-language telenovela produced by the American-based television network Telemundo.
Original release dates are October 20, 2008 – July 16, 2009.

In [ ]:
plot(plt.gca(),'Analia','F')

### Yamilex

[Como Tu, Ninguna](https://en.wikipedia.org/wiki/Como_t%C3%BA,_ninguna) (1995) is a Venezuelan telenovela that was produced by and seen on Venevisión. This telenovela lasted 281 episodes and was distributed internationally by Venevisión International. Original release dates are December 7, 1994 – December 7, 1995. One of the characters was named Yamilex.

In [ ]:
plot(plt.gca(),'Yamilex','F')

### Miley, Mylee, Mylie

[Miley Ray Cyrus](https://en.wikipedia.org/wiki/Miley_Cyrus) is an American singer, songwriter, and actress... Cyrus became a teen idol for her portrayal of the starring character Miley Stewart in the Disney Channel television series Hannah Montana in 2006... Her debut studio album Meet Miley Cyrus (2007) was certified triple-platinum by the Recording Industry Association of America (RIAA) for exceeding three million shipments. 

In [ ]:
fig,ax = plt.subplots(3,1,sharex=True,sharey=False)
plot(ax[0], 'Miley','F')
plot(ax[1], 'Mylee','F')
plot(ax[2], 'Mylie','F')
ax[0].set_xlim(1990,2015)